In [1]:
import rpy2

In [2]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr
from pandas import read_csv
import pandas as pd

pandas2ri to convert dataframes back and forth in R and python dataframes 

useful: when you want to load a R dataframe and then convert to pandas df or vice-versa

In [3]:
from rpy2.robjects import pandas2ri

In [4]:
utils= importr('utils')

Installing required packages from R in rpy2 to use function

In [5]:
packnames = ('stringdist','base')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

Movies Dataset

In [6]:
movies= read_csv('/Users/raj/Desktop/stringdist/movies.txt')

In [7]:
movies.head()

,title,gener,ratings
0,The Shawshank Redemption,Crime. Drama,9.3
1,The Godfather,Crime. Drama,9.2
2,The Dark Knight,Action. Crime. Drama,9.0
3,The Godfather: Part II,Crime. Drama,9.0
4,Pulp Fiction,Crime. Drama,8.9


In [8]:
messy_movie_titles= read_csv('/Users/raj/Desktop/stringdist/user_queries.txt')

In [9]:
messy_movie_titles.columns = ['messy_title']

In [10]:
messy_movie_titles.head()

,messy_title
0,dhe Shwshnk Redempdion
1,dhe Godfdher
2,dhe Drk Knighd
3,dhe Godfdher: Prd II
4,Pulp Ficdion


In [11]:
result=pd.concat([movies['title'],messy_movie_titles['messy_title']], axis=1,ignore_index=False)
result.columns = ['title','messy_title']

In [12]:
result.head()

,title,messy_title
0,The Shawshank Redemption,dhe Shwshnk Redempdion
1,The Godfather,dhe Godfdher
2,The Dark Knight,dhe Drk Knighd
3,The Godfather: Part II,dhe Godfdher: Prd II
4,Pulp Fiction,Pulp Ficdion


Calculating distance between strings using stringdist R package with methods Levenshtein ,Cosine And Jaccard Distance 

importing stringdist package in python using importr function

In [13]:
stringdist = importr('stringdist') 


Passing Levenshtein Method to stringdist

In [14]:
type(result)

pandas.core.frame.DataFrame

Need to convert this pandas dataframe to R data frame to pass to stringdist function

In [15]:
pandas2ri.activate()
robjects.globalenv['result'] = result

In [16]:
ld =stringdist.stringdist(result['title'], result['messy_title'], method='lv')


In [17]:
result['Levenshtein_distance'] = 0 
result['Levenshtein_distance'] = ld 

In [18]:
cd =stringdist.stringdist(result['title'], result['messy_title'], method='cosine',q=2)
result['Cosine_distance'] = 0 
result['Cosine_distance'] = cd 

In [19]:
jd =stringdist.stringdist(result['title'], result['messy_title'], method='jaccard',q=2)
result['Jaccard_distance'] = 0 
result['Jaccard_distance'] = jd 

In [20]:
result.head()

,title,messy_title,Levenshtein_distance,Cosine_distance,Jaccard_distance
0,The Shawshank Redemption,dhe Shwshnk Redempdion,4.0,0.301703,0.407407
1,The Godfather,dhe Godfdher,3.0,0.309934,0.461538
2,The Dark Knight,dhe Drk Knighd,3.0,0.258751,0.411765
3,The Godfather: Part II,dhe Godfdher: Prd II,5.0,0.347826,0.520000
4,Pulp Fiction,Pulp Ficdion,1.0,0.181818,0.307692
